In [1]:
import numpy as np
# import matplotlib.pyplot as plt # Already imported

from keras.datasets import boston_housing
Train, test = boston_housing.load_data(seed= 111)
Training_data, labels = Train[0], Train[1]
Test_data, test_labels = test[0], test[1]

57026/57026 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step


In [4]:
## Question 1
## Enter your solution here
print(Training_data.shape[0])

404


In [3]:
## Question 2
## Enter your solution here
print(Test_data.shape[0])

102


In [5]:
## Question 3
## Enter your solution here
print(Training_data.shape[1])

13


In [6]:
## Task: Add the dummy feature
## Enter your solution here

# Get the number of training and test samples
n_train = Training_data.shape[0]
n_test = Test_data.shape[0]

# Add a column of ones (the dummy feature) to the training data
# The resulting shape will be (n, d+1)
Training_data_with_bias = np.column_stack((np.ones(n_train), Training_data))

# Add a column of ones (the dummy feature) to the test data
Test_data_with_bias = np.column_stack((np.ones(n_test), Test_data))

# Also reshape labels to (n, 1) vector for consistent matrix operations
y_train = labels.reshape(-1, 1)
y_test = test_labels.reshape(-1, 1)

# Store the final design matrices for convenience:
X_train = Training_data_with_bias
X_test = Test_data_with_bias

In [8]:
## Calculation for Q4, Q5, Q6, Q7 (Normal Equation Solution)

# 1. Calculate X_train^T * X_train
XTX = X_train.T @ X_train

# 2. Calculate (X_train^T * X_train)^-1
XTX_inv = np.linalg.inv(XTX)

# 3. Calculate X_train^T * y_train
XTy = X_train.T @ y_train

# 4. Calculate optimal weights w*
w_star = XTX_inv @ XTy

# Calculate predictions for training and test sets
y_hat_train = X_train @ w_star
y_hat_test = X_test @ w_star

# Define the Loss function (RMSE)
def rmse_loss(y_true, y_pred):
    n = y_true.shape[0]
    # Sum of squared errors (SSE)
    sse = np.sum((y_true - y_pred)**2)
    # Mean squared error (MSE)
    mse = sse / n
    # Root mean squared error (RMSE)
    return np.sqrt(mse)

In [9]:
## Question 4
## Enter your solution here
# Sum of all weights in w*
sum_of_w_star = np.sum(w_star)
print(sum_of_w_star)

20.524582464661734


In [10]:
avg_predictions = np.mean(y_hat_train)
print(avg_predictions)

22.309158415840525


In [11]:
## Question 6
## Find the loss for the training data points using the above model (Normal Equation solution).
# Uses w_star and y_hat_train calculated in the setup for Q4-Q7
train_rmse_normal_eq = rmse_loss(y_train, y_hat_train)
print(train_rmse_normal_eq)

4.552387969840813


In [12]:
## Question 7
## Find the loss for the test data points using the above model (Normal Equation solution).
# Uses w_star and y_hat_test calculated in the setup for Q4-Q7
test_rmse_normal_eq = rmse_loss(y_test, y_hat_test)
print(test_rmse_normal_eq)

5.327662216181203


In [13]:
## Calculation for Q8, Q9, Q10 (Gradient Descent)

# Assuming previous variables: X_train, y_train, n_train
eta = 1e-10  # Learning rate
iterations = 100
d_plus_1 = X_train.shape[1] 

# Initialize weights as zero vector
w_gd = np.zeros((d_plus_1, 1))

# Gradient Descent Loop
for i in range(iterations):
    # 1. Calculate predictions (y_hat)
    y_hat = X_train @ w_gd
    
    # 2. Calculate the residual (error)
    residual = y_hat - y_train
    
    # 3. Calculate the gradient: (1/n) * X^T * residual
    gradient = (1 / n_train) * (X_train.T @ residual)
    
    # 4. Update the weights
    w_gd = w_gd - eta * gradient
    
# Predictions using the final weights
y_hat_train_gd = X_train @ w_gd
y_hat_test_gd = X_test @ w_gd

In [14]:
## Question 8
## Enter your solution here
sum_of_w_gd = np.sum(w_gd)
print(sum_of_w_gd)

0.00019673280976900844


In [15]:
## Question 9
## Enter your solution here
train_rmse_gd = rmse_loss(y_train, y_hat_train_gd)
print(train_rmse_gd)

24.10340179696994


In [16]:
## Question 10
## Enter your solution here
test_rmse_gd = rmse_loss(y_test, y_hat_test_gd)
print(test_rmse_gd)

24.94764717417876


In [17]:
## Calculation for Q11, Q12, Q13 (Stochastic Gradient Descent / Mini-Batch)

# Assuming previous variables: X_train, y_train, n_train, X_test
eta_sgd = 1e-8  # Learning rate
iterations_sgd = 1000
d_plus_1 = X_train.shape[1] 

# Calculate batch size: ceil(n_train / 5)
batch_size = int(np.ceil(n_train / 5)) # n_train/5 = 404/5 = 80.8 -> batch_size = 81

# Initialize weights as zero vector
w_sgd = np.zeros((d_plus_1, 1))

# SGD Loop
for i in range(1, iterations_sgd + 1):
    # Set seed for random sampling (seed at i)
    np.random.seed(i)
    
    # Randomly sample 'batch_size' indices
    batch_indices = np.random.choice(n_train, batch_size, replace=False)
    
    # Create the mini-batch
    X_batch = X_train[batch_indices, :]
    y_batch = y_train[batch_indices, :]
    
    # 1. Calculate predictions (y_hat) on the batch
    y_hat_batch = X_batch @ w_sgd
    
    # 2. Calculate the residual (error) on the batch
    residual_batch = y_hat_batch - y_batch
    
    # 3. Calculate the gradient: (1/batch_size) * X_batch^T * residual_batch
    gradient_sgd = (1 / batch_size) * (X_batch.T @ residual_batch)
    
    # 4. Update the weights
    w_sgd = w_sgd - eta_sgd * gradient_sgd
    
# Final weights after 1000 iterations are in w_sgd
# Predictions using the final weights
y_hat_train_sgd = X_train @ w_sgd
y_hat_test_sgd = X_test @ w_sgd

In [18]:
## Question 11
## Enter the sum of all the weights (w_sgd)
sum_of_w_sgd = np.sum(w_sgd)
print(sum_of_w_sgd)

0.061334305432517444


In [19]:
## Question 12
## Enter your solution here
train_rmse_sgd = rmse_loss(y_train, y_hat_train_sgd)
print(train_rmse_sgd)

10.899808528977173


In [20]:
## Question 13
## Enter your solution here
test_rmse_sgd = rmse_loss(y_test, y_hat_test_sgd)
print(test_rmse_sgd)

10.663599407070361


In [23]:
import matplotlib.pyplot as plt
## Question 14
## Plot the scatter plot between feature and the labels. Enter your answer as 0.
plt.figure(figsize=(8, 5))
# X has shape (n, 1), y has shape (n, 1)
plt.scatter(X, y, s=5) 
plt.title('Scatter Plot of Feature vs. Label')
plt.xlabel('Feature (X)')
plt.ylabel('Label (y)')
plt.show()

# Enter your answer as 0
print(0)

NameError: name 'X' is not defined

<Figure size 800x500 with 0 Axes>

In [24]:
## Question 15
## How many examples are there in the training dataset?
print(X.shape[0])

NameError: name 'X' is not defined